In [3]:
import pandas as pd

nasdaq = pd.read_csv('nasdaq_screen.csv')
nasdaq.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$122.31,-1.170,-0.948%,3.620872e+10,United States,1999.0,1733059,Industrials,Electrical Products
1,AA,Alcoa Corporation Common Stock,$34.47,-1.040,-2.929%,6.202021e+09,NaN,2016.0,7271376,Industrials,Metal Fabrications
2,AAC,Ares Acquisition Corporation Class A Ordinary ...,$9.92,0.000,0.00%,1.240000e+09,NaN,2021.0,425413,Finance,Business Services
3,AACG,ATA Creativity Global American Depositary Shares,$1.72,-0.015,-0.865%,5.434957e+07,China,2008.0,11633,Consumer Discretionary,Service to the Health Industry
4,AACI,Armada Acquisition Corp. I Common Stock,$9.93,0.020,0.202%,2.056453e+08,United States,2021.0,454,Industrials,Consumer Electronics/Appliances


In [4]:
# download
DOWNLOAD = False
if DOWNLOAD:

    nasdaq = nasdaq.sort_values(by="Market Cap", ascending=False)
    nasdaq.head()
    # get all the data for them since 2010
    import yfinance as yf
    from pathlib import Path


    # keep only X biggest
    step = 1000
    for i in range(0,5000, step):
        filename = "data/nasdaq_prices_%s.csv.gz" % str(i+1)
        my_file = Path(filename)
        if not my_file.is_file():
            nasdaq_step = nasdaq.iloc[i:i+step]
            nasdaq_prices = yf.download(nasdaq_step['Symbol'].dropna().tolist(), start='2010-01-01', end='2022-12-31')
            # nasdaq_prices = nasdaq_prices.fillna(method="bfill")
            # nasdaq_prices = nasdaq_prices.fillna(method="ffill")
            # nasdaq_prices.index.levels[0].columns = nasdaq_prices.index.levels[0].columns.astype(str)
            # nasdaq_prices.index.levels[1].columns = nasdaq_prices.index.levels[1].columns.astype(str)
            print("shape: ", nasdaq_prices.shape)
            nasdaq_prices.to_csv(filename, compression = 'gzip')


In [5]:
! ls -lh /data

ls: cannot access '/data': No such file or directory


In [6]:
import glob
data = []
for file in glob.glob('./data/*.csv.gz'):
    print(file)
    data.append(pd.read_csv(file, index_col=0, header=[0,1], parse_dates=True, compression = 'gzip'))
data = pd.concat(data, axis=1)

# data.index = data.index.set_levels(data.index.levels[0].astype(str), level=0)
# data.index = data.index.set_levels(data.index.levels[1].astype(str), level=1)
# data.to_parquet("data/nasbig.parquet")
print(data.shape)
print(data.head())

./data/nasdaq_prices_1001.csv.gz
./data/nasdaq_prices_2001.csv.gz
./data/nasdaq_prices_3001.csv.gz
./data/nasdaq_prices_1.csv.gz
(3220, 24000)
           Adj Close                                                   \
                AAON       AAWW         AB      ABCB ABCL ABCM    ABG   
Date                                                                    
2009-12-31  5.154801  37.250000  10.276358  6.426569  NaN  NaN  11.53   
2010-01-04  5.287043  38.200001  10.386068  6.561203  NaN  NaN  11.83   
2010-01-05  5.133642  40.310001  10.375102  6.633008  NaN  NaN  12.39   
2010-01-06  4.929989  38.630001  10.378757  6.767643  NaN  NaN  12.76   
2010-01-07  5.112483  38.950001  10.386068  6.992035  NaN  NaN  13.05   

                                     ...    Volume                          \
                  ABM       ABR ACA  ...      ZBRA ZEN  ZG  ZI        ZION   
Date                                 ...                                     
2009-12-31  15.783669  0.815225 NaN  .

In [7]:
closes = data["Adj Close"]

# closes = closes.fillna(method="bfill")
# closes = closes.fillna(method="ffill")

## setting up a baseline 

1. equal invest into all back then to now
2. sharpe invest back then and hold
3. yearly sharpe rebalance

In [8]:
closes.head()

,AAON,AAWW,AB,ABCB,ABCL,ABCM,ABG,ABM,ABR,ACA,...,ZBRA,ZEN,ZG,ZI,ZION,ZM,ZNH,ZS,ZTO,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2009-12-31,5.154801,37.250000,10.276358,6.426569,NaN,NaN,11.53,15.783669,0.815225,NaN,...,28.350000,NaN,NaN,NaN,11.146583,NaN,12.823545,NaN,NaN,NaN
2010-01-04,5.287043,38.200001,10.386068,6.561203,NaN,NaN,11.83,16.242056,0.807032,NaN,...,28.670000,NaN,NaN,NaN,11.580979,NaN,13.289250,NaN,NaN,NaN
2010-01-05,5.133642,40.310001,10.375102,6.633008,NaN,NaN,12.39,16.122883,0.786549,NaN,...,28.620001,NaN,NaN,NaN,11.989312,NaN,13.505470,NaN,NaN,NaN
2010-01-06,4.929989,38.630001,10.378757,6.767643,NaN,NaN,12.76,16.122883,0.876674,NaN,...,28.400000,NaN,NaN,NaN,13.031859,NaN,13.638530,NaN,NaN,NaN
2010-01-07,5.112483,38.950001,10.386068,6.992035,NaN,NaN,13.05,16.330469,0.921737,NaN,...,27.690001,NaN,NaN,NaN,14.491426,NaN,13.563684,NaN,NaN,NaN


In [9]:
# 1. equal invest

startMoney = 10000
moneyPerStock = startMoney / len(closes.columns)
print("going to invest %.2f$ per stock" % moneyPerStock)
equalNrStocks = moneyPerStock / closes.iloc[0].dropna() 
print(equalNrStocks.sort_values(ascending=False).head(10))

going to invest 2.50$ per stock
SCWO    6.250000
VTNR    5.208333
ZYXI    5.191273
SIRI    4.413108
CPRX    3.968254
AMEH    3.571429
MDXG    3.424657
NEWP    3.086420
ABR     3.066638
NSSC    2.994012
Name: 2009-12-31 00:00:00, dtype: float64


In [10]:
# at the end
win = equalNrStocks * closes.iloc[-1]
print(win.sort_values(ascending=False).head(10))

LNG     169.580854
TPL     161.971698
AMEH    142.071427
NXST    123.699904
NOVT    116.752878
RGEN    114.823600
SLP     114.581483
DPZ     113.632996
PATK    106.976309
DXCM    103.258990
dtype: float64


In [11]:
## total win
print("total equal invest win is: %.2f$" % win.sum())

total equal invest win is: 22370.25$


## get a sharpe baseline

In [12]:
from pypfopt.efficient_frontier import EfficientFrontier
from datetime import datetime
from pypfopt import risk_models
from pypfopt import expected_returns
# make the estimation from then 

subcloses = closes[:datetime(2015,1,1)]
subcloses = subcloses.dropna(axis=1, how="all")

# first step: get highest gainers
gains = subcloses.iloc[-1] / subcloses.iloc[0]
gains = gains.sort_values(ascending=False)
print(gains.head(10))

# take only x highest gainers
subcloses = subcloses[gains.index[:200]]

subcloses = subcloses.fillna(method="ffill")
subcloses = subcloses.fillna(method="bfill")

# print(nasclose.describe())
assert not subcloses.isnull().values.any()
assert len(subcloses) > 0
print(subcloses.shape)
# print("something nan`", nasclose.isnull().values.any())
# print(nasclose.describe())
mu = expected_returns.mean_historical_return(subcloses)
S = risk_models.sample_cov(subcloses)
# contains errors
mu = mu.dropna()
subcloses = subcloses[mu.index]
mu = expected_returns.mean_historical_return(subcloses)
S = risk_models.sample_cov(subcloses)

assert not mu.isnull().values.any()
assert not S.isnull().values.any()


# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
weights = dict(ef.max_sharpe())
# weights = dict(efcdar.min_cdar())
print(weights)

LNG     29.090909
ACAD    24.053029
JAZZ    20.777918
PATK    18.098762
REGN    16.966501
TARO    16.688063
MDXG    15.794520
ACHC    13.542035
NXST    13.209751
DPZ     12.461288
dtype: float64
(1262, 200)
{'LNG': 7.5509455551e-06, 'ACAD': 0.0029877239896751, 'JAZZ': 0.0249220634513196, 'PATK': 0.0055229124894073, 'REGN': 0.0151268857846902, 'TARO': 0.1693922717596115, 'MDXG': 0.0233405906849479, 'ACHC': 0.0442249190007878, 'NXST': 6.1199393953e-06, 'DPZ': 0.0726766690650664, 'PTSI': 0.0424449477647714, 'LAD': 9.6509615836e-06, 'AAL': 6.4433770102e-06, 'BHC': 0.0214478777299543, 'VRTS': 6.3223148407e-06, 'IDT': 6.4026972521e-06, 'URI': 1.03735930993e-05, 'UAA': 5.1740839205e-06, 'AMOT': 0.0309363688972999, 'SBGI': 8.7044547149e-06, 'VTNR': 0.0070203592813362, 'DDD': 1.30105722303e-05, 'BVH': 0.0112926154137204, 'WETF': 2.2127464905e-06, 'IMOS': 6.5033218151e-06, 'NBIX': 7.2832937403e-06, 'EXAS': 6.4229865281e-06, 'INCY': 5.6600689086e-06, 'CVI': 6.6943025625e-06, 'XPO': 0.000839040862

In [13]:
from datetime import date
# simulate that fuck 
# 2015 - 2022
# find i position where date index of closes is 2015
start_i = len(subcloses)

startMoney = 10000
nrStocks = dict()
for stock, weight in weights.items():
    # print(stock, weight)
    nrStocks[stock] = startMoney * weight / closes[stock].iloc[start_i]
# sell
win = dict()
for stock, weight in weights.items():
    win[stock] = nrStocks[stock] * closes[stock].iloc[-1]
    
# print(win)
print("total win is: %.2f$" % sum(win.values()))
win = pd.Series(win)
win = win.sort_values(ascending=False)
print("best positions: ")
print(win.head(20))

total win is: 22302.56$
best positions: 
TPL     4400.438425
SLP     3387.867019
DPZ     2647.544682
ORLY    1796.191325
PPBI    1239.584885
PTSI    1012.265888
HBNC     944.680136
MBWM     870.803540
TCX      764.284953
MMP      736.112200
AMOT     626.655013
ACHC     590.378339
DLTR     532.891926
TARO     401.301181
DJCO     289.534266
REGN     260.162069
JAZZ     196.625532
SPNS     191.823162
HCI      189.001546
NFLX     177.229892
dtype: float64


In [14]:
# just nasdaq
import yfinance as yf
naspure = yf.download("^IXIC", start='2010-01-01', end='2022-12-31')
nrStocks = startMoney / naspure["Adj Close"].iloc[0]
win = nrStocks * naspure["Adj Close"].iloc[-1]
print("just buying and holding nasdaq would have been: ", win)

[*********************100%***********************]  1 of 1 completed
just buying and holding nasdaq would have been:  48703.876447397895


In [15]:
# calculate best possible case
cleancloses = closes.fillna(method="ffill")
cleancloses = cleancloses.fillna(method="bfill")
cleancloses = cleancloses.dropna(axis=1, how="all")
# any nan in there?

allwin = cleancloses.iloc[-1] / cleancloses.iloc[0] 

biggestGainers = allwin.sort_values(ascending=False).head(5)
print(biggestGainers)

PRTA           inf
TSLA    180.709464
LNG      67.832342
TPL      64.788679
AMEH     56.828571
dtype: float64


In [16]:
# best possible case would have been tesla
# with * 180 the invest
teslaWin = biggestGainers["TSLA"] * 10000 # startmoney
print("best possible case with TSLA would have been %.2f$" % teslaWin)

best possible case with TSLA would have been 1807094.64$


## baseline compared:

- equal invest in 2010: 
    - 22370.25$
    - + 12.360$; + 123%
    - + 10 Pct per year
- sharpe ab 2015
    - 22302.53$
    - + 12.370$; + 123%
    - + 10 pct per year
- just hold nasdaq
    - 48703$
    - 38.703$ win; +387%
    - + 32% per year
- best possible case (TSLA)
    - 1.807.094.64$
    - + 18.070% win
    - + 1.500% win per year

## lightgbm: 

train the model to predict the final multiplier

In [17]:
y = allwin.sort_values(ascending=False)[1:] # lazy drop of PRTA inf
y

TSLA    180.709464
LNG      67.832342
TPL      64.788679
AMEH     56.828571
NXST     49.479962
           ...    
EGLE      0.000830
GEVO      0.000024
CIO      -7.526252
AIV     -44.603280
ITRN    -58.510083
Length: 3992, dtype: float64

In [18]:
data.head()

Adj Close                                                   \
                AAON       AAWW         AB      ABCB ABCL ABCM    ABG   
Date                                                                    
2009-12-31  5.154801  37.250000  10.276358  6.426569  NaN  NaN  11.53   
2010-01-04  5.287043  38.200001  10.386068  6.561203  NaN  NaN  11.83   
2010-01-05  5.133642  40.310001  10.375102  6.633008  NaN  NaN  12.39   
2010-01-06  4.929989  38.630001  10.378757  6.767643  NaN  NaN  12.76   
2010-01-07  5.112483  38.950001  10.386068  6.992035  NaN  NaN  13.05   

                                     ...    Volume                          \
                  ABM       ABR ACA  ...      ZBRA ZEN  ZG  ZI        ZION   
Date                                 ...                                     
2009-12-31  15.783669  0.815225 NaN  ...  184800.0 NaN NaN NaN   3239800.0   
2010-01-04  16.242056  0.807032 NaN  ...  168800.0 NaN NaN NaN   3974600.0   
2010-01-05  16.122883  0.786549 NaN  ...  168800.0 NaN NaN NaN   5605500.0   
2010-01-06  16.122883  0.876674 NaN  ...  385300.0 NaN NaN NaN  12615200.0   
2010-01-07  16.330469  0.921737 NaN  ...  183600.0 NaN NaN NaN  24716800.0   

                                     
            ZM      ZNH  ZS ZTO ZTS  
Date                                 
2009-12-31 NaN  48900.0 NaN NaN NaN  
2010-01-04 NaN  48200.0 NaN NaN NaN  
2010-01-05 NaN  61400.0 NaN NaN NaN  
2010-01-06 NaN  85600.0 NaN NaN NaN  
2010-01-07 NaN  29500.0 NaN NaN NaN  

[5 rows x 24000 columns]

In [23]:
data = data.swaplevel(1,0, axis=1)
data.head()

,AAON,AAWW,AB,ABCB,ABCL,ABCM,ABG,ABM,ABR,ACA,...,ZBRA,ZEN,ZG,ZI,ZION,ZM,ZNH,ZS,ZTO,ZTS
,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,...,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume
Date,,,,,,,,,,,,,,,,,,,,,
2009-12-31,5.154801,37.250000,10.276358,6.426569,NaN,NaN,11.53,15.783669,0.815225,NaN,...,184800.0,NaN,NaN,NaN,3239800.0,NaN,48900.0,NaN,NaN,NaN
2010-01-04,5.287043,38.200001,10.386068,6.561203,NaN,NaN,11.83,16.242056,0.807032,NaN,...,168800.0,NaN,NaN,NaN,3974600.0,NaN,48200.0,NaN,NaN,NaN
2010-01-05,5.133642,40.310001,10.375102,6.633008,NaN,NaN,12.39,16.122883,0.786549,NaN,...,168800.0,NaN,NaN,NaN,5605500.0,NaN,61400.0,NaN,NaN,NaN
2010-01-06,4.929989,38.630001,10.378757,6.767643,NaN,NaN,12.76,16.122883,0.876674,NaN,...,385300.0,NaN,NaN,NaN,12615200.0,NaN,85600.0,NaN,NaN,NaN
2010-01-07,5.112483,38.950001,10.386068,6.992035,NaN,NaN,13.05,16.330469,0.921737,NaN,...,183600.0,NaN,NaN,NaN,24716800.0,NaN,29500.0,NaN,NaN,NaN


In [34]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

lightgdm_models = dict()

X = []
Y = []

STOCKS 

for year in range(2010,2020):
    start = datetime(year,1,1)
    end = datetime(year+1,12,31)
    futureend = datetime(year+2,12,31)
    subdata = data[start:end]
    for stock in ["TSLA"]:
        futurePrice = data[end:futureend][stock]["Adj Close"].iloc[-1]
        print(futurePrice)
        subdatastock = subdata[stock]
        # nextyearprice = subdatastock["Adj Close"].iloc[-1]
        y = futurePrice / subdatastock["Adj Close"].iloc[-1] # should predict how much it will grow
        X.append(subdatastock)
        Y += [y] * len(subdatastock)
        
for stock in 
X = pd.concat(X)
Y = pd.DataFrame(Y)
print(Y)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, shuffle = True)
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

# fit
params = {
    'task': 'train', 
    'boosting': 'gbdt',
    'objective': 'regression',
    'num_leaves': 10,
    'learnnig_rage': 0.05,
    'metric': {'l2','l1'},
    'verbose': -1
}
 
model = lgb.train(params,
        train_set=lgb_train,
        # valid_sets=lgb_eval,
        early_stopping_rounds=30)



2.257999897003174
10.028667449951172
14.827333450317385
16.000667572021484
14.246000289916992
20.75666618347168
22.18666648864746
27.8886661529541
235.22332763671875
352.260009765625
287.8099975585937
             0
0     1.185924
1     1.185924
2     1.185924
3     1.185924
4     1.185924
...        ...
5554  0.817039
5555  0.817039
5556  0.817039
5557  0.817039
5558  0.817039

[5559 rows x 1 columns]
[1]	valid_0's l1: 1.51425	valid_0's l2: 4.54013
Training until validation scores don't improve for 30 rounds
[2]	valid_0's l1: 1.48967	valid_0's l2: 4.40992
[3]	valid_0's l1: 1.46854	valid_0's l2: 4.30443
[4]	valid_0's l1: 1.44908	valid_0's l2: 4.22019
[5]	valid_0's l1: 1.43269	valid_0's l2: 4.15733
[6]	valid_0's l1: 1.41627	valid_0's l2: 4.09853
[7]	valid_0's l1: 1.40276	valid_0's l2: 4.05331
[8]	valid_0's l1: 1.38943	valid_0's l2: 4.01668
[9]	valid_0's l1: 1.37789	valid_0's l2: 3.98638
[10]	valid_0's l1: 1.37048	valid_0's l2: 3.95593
[11]	valid_0's l1: 1.36371	valid_0's l2: 3.93502
[12

/home/jguese/miniconda3/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
